## Aggregate game information from multiple sources

In this script, we bring together a bunch of information about games that are available on Steam and save them in a database. The two main sources of information are an unofficial API called 'steampowered', as well web scraping the steam store. 

We are interested in the following information: 
- Each games web store app ID
- A detailed description of each game
- The game developers (ed, 'Valve')
- The initial and final price of each game
- The game title
- Game genre (eg, 'Action'))
- A metacritic score for each game
- How many recommendations has the game received
- The games release date
- Tags for each game, of which each game typically has around 10 (eg, 'FPS', 'Simulation', 'Racing')


In [ ]:
import pickle
import requests
import json
import re
import datetime
import dateutil.parser
from forex_python.converter import CurrencyRates
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2

currentRates = CurrencyRates() # Get current curreny exchange rates

# Load in previously sorted game counts. 
with open('gameCounts.pickle', 'rb') as inputfile:
    gameCounts = pickle.load(inputfile)

## Collect game information

In the function below, we first query gameinformation from the 'steampowered' API, and then use BeautifulSoup to scrape the Steam web store for tags. Al of this information is then stored in a pandas dataframe. 

In [ ]:
# Define funciton to query game information and return feature dict
def grabGameInfo(gameID):
    # Query game info
    currentRequest = requests.get("http://store.steampowered.com/api/appdetails?appids={}".format(gameID))
    try:
        jsonIndex   = re.search(r'\d+', currentRequest.text).group()
        json2pyData = json.loads(currentRequest.content)
        try:
            
            gameData = json2pyData[jsonIndex]['data'] # all info in json nested format
            gameKeys = gameData.keys()
            tempDict = dict()
            tempDict['gameName']            = gameData['name']
            tempDict['type']                = gameData['type']
            tempDict['detailedDescription'] = gameData['detailed_description']
            tempDict['developers']          = gameData['developers'][0]
            tempDict['genre']               = gameData['genres'][0]['description']
            tempDict['is_free']             = gameData['is_free']

            # Control for the case that certain fields are not present
            if 'metacritic' in gameKeys:
                tempDict['metacriticScore'] = gameData['metacritic']['score']
            else: 
                tempDict['metacriticScore'] = 'Empty'
            if 'recommendations' in gameKeys:
                tempDict['recommendations'] = gameData['recommendations']['total']
            else:
                tempDict['recommendations'] = 'Empty' 

            # The code below loads in the date of the game release and converts it into an integer with format %YYYY%MM%DD
            #release_date = datetime.datetime.strptime(gameData['release_date']['date'], '%b %d, %Y')
            try:
                release_date = dateutil.parser.parse(gameData['release_date']['date'])
                yearStr  = str(release_date.year)
                monthStr = str(release_date.month)
                dayStr   = str(release_date.day)
                if len(monthStr) == 1:
                    monthStr = "0" + monthStr
                if len(dayStr) == 1:
                    dayStr = "0" + dayStr
                tempDict['releaseDate'] = int(yearStr + monthStr + dayStr)  
            except:
                tempDict['releaseDate'] = 'Empty'

            # Get the price of the game and convert it to US dollars if it is in another currency
            if 'price_overview' in gameKeys:
                currency = gameData['price_overview']['currency']
                if currency != 'USD':
                    init_price  = (gameData['price_overview']['initial'] * currentRates.get_rates('USD')[currency])/100
                    final_price = (gameData['price_overview']['final'] * currentRates.get_rates('USD')[currency])/100
                else:
                    init_price  = (gameData['price_overview']['initial'])/100
                    final_price = (gameData['price_overview']['final'])/100
                tempDict['init_price']  = init_price
                tempDict['final_price'] = final_price
            else: # If there is no price information
                tempDict['init_price'] = 'Empty'
                tempDict['final_price'] = 'Empty'

            # Now let's scrape the steam store website to obtain the specific tags for each game    
            url  = "http://store.steampowered.com/app/{}/".format(str(gameID)) # URL for game
            page = urlopen(url) 
            soup = BeautifulSoup(page,'lxml') # scrape page
            links = soup.find_all("a") # The tag information is contained in some of the links on the page    

            tags = [] # initialize tags list
            for link in links:
                            # We try this because some links don't have have a field 'class' 
                try:
                    if link.get("class")[0] == "app_tag":
                        linkContents = link.contents[0]
                        tags.append(re.sub('\s+', '', linkContents))
                except:
                    pass    
            tempDict['tags'] = tags
            
        except:
            return('Empty')


        print(tempDict['gameName'] ,end = "\r")
    except:
        return('Empty')
    
    return(tempDict)

numGames = 20000 # number of games to query
gameDict = {str(gameID): grabGameInfo(gameID) for gameID, count in gameCounts.most_common(numGames)}
print('Done!')

## Save game information

In [ ]:
# Now let's save the dict as a json file 
jsonFile = json.dumps(gameDict)
f = open("gameDict_20k.json","w")
f.write(jsonFile)
f.close()

In [ ]:
# Load game dict back in if we need to
with open('gameDict_20k.json') as json_data:
    gameDict = json.load(json_data)
# load game data into a dataframe
gameDataframe = pd.DataFrame(gameDict)
gameDataframe = gameDataframe.T # transpose to get 

## Connect to PostgreSQL database

In [ ]:
# Define SQL database info
db_name  = 'UserInfo'
username = 'username'
host     = 'localhost'
pwd      = 'password'
port     = '5432'
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(username, pwd, host, port, db_name))
print(engine.url)

# connect to the database:
con = None
con = psycopg2.connect(database = db_name, user = username, password = pwd, host = host)
cur = con.cursor() # get a cursor to our current connection

In [ ]:
# For writing the game data into SQL we want to replace 'Empty' entries with NaN's
gameDataframe = gameDataframe.replace('Empty',np.nan)
# Then we write all the game data into a .csv file
gameDataframe.to_csv("/home/iain/Documents/InsghtProject/allGames.csv")

## Copy game data into the database

Similar to the user information, we would like to save game information into a relational database to make it easier to query and interact with. In the cell below we first initialize a data table in the database, and then copy our game data into the table. 

In [ ]:
# make a query to initialize the allGames table in postgres.
# Then we copy our data from the csv file into the table we just created. 
# These changes have to be committed, otherwise thy will not be permanent
# Finally, I query some data from the tabe, since the read_sql function expects an output.
create_table_sql = """
CREATE TABLE allgames
(
  gameID numeric NULL,
  detaileDescription text NULL,
  developers text NULL,
  final_price numeric NULL,
  gameName text NULL,
  genre text NULL,
  init_price numeric NULL,
  is_free text NULL,
  metacriticScore numeric NULL,
  recommendations numeric NULL,
  releaseDate numeric NULL,
  tags text NULL,
  type text NULL
);

COPY allgames FROM '/home/iain/Documents/InsghtProject/allGames.csv' WITH DELIMITER ',' HEADER CSV;

COMMIT;

SELECT * FROM allgames WHERE final_price = 9.99;
"""
# Run the above command using postgres
gameInfo = pd.read_sql_query(create_table_sql,con)